In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import container
from matplotlib.lines import Line2D
from matplotlib.collections import PolyCollection
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
%matplotlib qt

In [2]:
full_data = pd.read_csv('../../Data/BHcompilation_updated.csv', encoding='ISO-8859-1')

In [3]:
# Data for fitting
data = full_data.loc[full_data.SELECTED == 1]

# Normalizing uncertainties to 3-sigma values
data.loc[data.CONFLEVEL == 1, 'DMBH'] = data.DMBH*3
data.loc[data.CONFLEVEL == 2, 'DMBH'] = data.DMBH*(3/2)

# Reset the indices to avoid KeyError's later on
data = data.reset_index(drop=True)

In [4]:
# fig = plt.figure(figsize=(10,8), layout='constrained')
# plt.xlabel(r'$\sigma$ log(km/s)', fontsize='x-large', fontweight='bold')
# plt.ylabel(r'$M_\bullet$ log(M$_\odot$)', fontsize='x-large', fontweight='bold')
# plt.scatter(data.SIG, data.MBH, label='data')
# plt.show()

In [5]:
# multipliers = 1
# iters = 1
# num_params = 4

# fitted_params = np.empty((1,4))

# for i in range(1, multipliers + 1):

#     # Loading data from .txt files
#     loaded_params = np.loadtxt('../Data/Params_test/params_batch_{}.txt'.format(i), delimiter=" ", dtype=float)
#     loaded_params = np.reshape(loaded_params, (-1, num_params)) # reshape for vectors of parameters

#     # Add to data array
#     fitted_params = np.vstack((fitted_params, loaded_params))


# # Remove random initialized first row
# fitted_params = fitted_params[1:, :]

# np.save('../Data/fitted_params_test_{}.npy'.format(str(int(iters*multipliers))), fitted_params)

In [6]:
# fitted_params = np.load('../Data/fitted_params_test_{}.npy'.format(str(int(iters*multipliers))))

# avg_params = np.median(fitted_params, axis=0)

# beta0 = avg_params[0]

# beta1 = avg_params[1]

# gamma0 = avg_params[2]

# gamma1 = avg_params[3]

In [7]:
# logistic function
def logit(x, beta0, beta1):
    return (1/(1+np.exp(-(beta0 + beta1*x))))
  
# log_linear function
def linear(x, gamma0, gamma1):
    return gamma0 + gamma1*x
  
# hurdle model functional form
def hurdle(x, beta0, beta1, gamma0, gamma1):
    return logit(x, beta0, beta1)*linear(x, gamma0, gamma1)

# hurdle model functional form
def hurdle_brms(x, beta0, beta1, gamma0, gamma1):
    return (1-logit(x, beta0, beta1))*linear(x, gamma0, gamma1)
# def percentile(x, q):

#     lower = (100-q)/2
#     upper = q + lower

#     lower = np.percentile(x, lower, axis=0)
#     upper = np.percentile(x, upper, axis=0)

#     return lower, upper

In [8]:
# lower_68, upper_68 = percentile(fitted_params, 68)
# lower_95, upper_95 = percentile(fitted_params, 95)

# # beta0_err = upper_68[0] - lower_68[0]
# # beta1_err = upper_68[1] - lower_68[1]
# # gamma0_err = upper_68[2] - lower_68[2]
# # gamma1_err = upper_68[3] - lower_68[3]

# beta0_err = upper_95[0] - lower_95[0]
# beta1_err = upper_95[1] - lower_95[1]
# gamma0_err = upper_95[2] - lower_95[2]
# gamma1_err = upper_95[3] - lower_95[3]

# print('95% CI for beta0: ', '(', lower_95[0], ',', upper_95[0], ')')
# print('95% CI for beta1: ', '(', lower_95[1], ',', upper_95[1], ')')
# print('95% CI for gamma0: ', '(', lower_95[2], ',', upper_95[2], ')')
# print('95% CI for gamma1: ', '(', lower_95[3], ',', upper_95[3], ')')

# print()

# print('68% CI for beta0: ', '(', lower_68[0], ',', upper_68[0], ')')
# print('68% CI for beta1: ', '(', lower_68[1], ',', upper_68[1], ')')
# print('68% CI for gamma0: ', '(', lower_68[2], ',', upper_68[2], ')')
# print('68% CI for gamma1: ', '(', lower_68[3], ',', upper_68[3], ')')

In [9]:
# Data omitted from fits
omitted_upplim = full_data.loc[(full_data.SELECTED != 1) & (full_data.UPPERLIMIT == 1)]
omitted_true = full_data.loc[(full_data.SELECTED != 1) & (full_data.UPPERLIMIT == 0)]

# Fit data
stars_upplim = full_data.loc[(full_data.SELECTED == 1) & (full_data.MBH > 0) & (full_data.UPPERLIMIT == 1) & ((full_data.TYPE == 'star') | (full_data.TYPE == 'stars'))]
stars_upplim_zero = full_data.loc[(full_data.SELECTED == 1) & (full_data.MBH == 0) & (full_data.UPPERLIMIT == 1) & ((full_data.TYPE == 'star') | (full_data.TYPE == 'stars'))]
stars_true = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 0) & ((full_data.TYPE == 'star') | (full_data.TYPE == 'stars'))]

gas_upplim = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 1) & ((full_data.TYPE == 'gas') | (full_data.TYPE == 'CO'))]
gas_true = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 0) & ((full_data.TYPE == 'gas') | (full_data.TYPE == 'CO'))]

reverb_upplim = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 1) & (full_data.TYPE == 'reverb')]
reverb_true = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 0) & (full_data.TYPE == 'reverb')]

maser_upplim = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 1) & (full_data.TYPE == 'maser')]
maser_true = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 0) & (full_data.TYPE == 'maser')]

In [10]:
# # Velocity dispersion (sigma) values for plotting
# sigma_plot_values = np.linspace(1, 2.8, num=100)

# # Calculating fit with confidence interval
# fitted_line = hurdle(sigma_plot_values, beta0, beta1, gamma0, gamma1)
# y_2016 = -4 + 5.35*sigma_plot_values # coefficients from Remco Van Den Bosch 2016
# linear_portion = linear(sigma_plot_values, gamma0, gamma1)

# # Plot labels
# fig = plt.figure(figsize=(10,8), layout='constrained')
# plt.xlim(1.0, 2.8)
# plt.ylim(-0.5, 11)

# # Plotting average fit with previous ones
# plt.plot(sigma_plot_values, fitted_line, color='black', label='Hurdle Fit')
# plt.plot(sigma_plot_values, linear_portion, color='red', linewidth=1.4, linestyle='dotted')
# #plt.plot(sigma_plot_values, hurdle(sigma_plot_values, -17.43, 9, 0.49, 0.7), color='black', linestyle='dashed', label='Without uncertainties') # coefficients from previous fit (without uncertainties)
# plt.plot(sigma_plot_values, y_2016, color='black', linestyle='dotted', label='van den Bosch 2016')
# plt.plot(sigma_plot_values, y_2016+0.49, color='dimgray', linestyle='dashed', alpha=0.7)
# plt.plot(sigma_plot_values, y_2016-0.49, color='dimgray', linestyle='dashed', alpha=0.7)
# plt.plot(sigma_plot_values, y_2016+3*0.49, color='dimgray', linestyle='dotted', alpha=0.7)
# plt.plot(sigma_plot_values, y_2016-3*0.49, color='dimgray', linestyle='dotted', alpha=0.7)

# # Plotting all fits for confidence interval
# for i in range(len(fitted_params)):
#   plt.plot(sigma_plot_values, hurdle(sigma_plot_values, fitted_params[i][0], fitted_params[i][1], fitted_params[i][2], fitted_params[i][3]), alpha=0.003, color='lightgrey')

# # Plotting data
# plt.errorbar(stars_upplim.SIG, stars_upplim.MBH, stars_upplim.DMBH, ls='', lw=0.75, color='darkred', marker='.')
# plt.scatter(stars_upplim.SIG, stars_upplim.MBH, color='darkred', marker='v', s=50)
# plt.errorbar(stars_true.SIG, stars_true.MBH, stars_true.DMBH, ls='', lw=0.75, color='darkred', marker='.', label='stars')
# plt.scatter(stars_true.SIG, stars_true.MBH, color='darkred', s=36)

# plt.scatter(stars_upplim_zero.SIG, stars_upplim_zero.MBH, color='darkred', marker='v', s=50)

# plt.errorbar(gas_upplim.SIG, gas_upplim.MBH, gas_upplim.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.')
# plt.scatter(gas_upplim.SIG, gas_upplim.MBH, color='dodgerblue', marker='v', s=50)
# plt.errorbar(gas_true.SIG, gas_true.MBH, gas_true.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.', label='gas')
# plt.scatter(gas_true.SIG, gas_true.MBH, color='dodgerblue', s=36)

# plt.errorbar(reverb_upplim.SIG, reverb_upplim.MBH, reverb_upplim.DMBH, ls='', lw=0.75, color='black', marker='.')
# plt.scatter(reverb_upplim.SIG, reverb_upplim.MBH, color='black', marker='v', s=50)
# plt.errorbar(reverb_true.SIG, reverb_true.MBH, reverb_true.DMBH, ls='', lw=0.75, color='black', marker='.', label='reverberation')
# plt.scatter(reverb_true.SIG, reverb_true.MBH, color='black', s=36)

# plt.errorbar(maser_upplim.SIG, maser_upplim.MBH, maser_upplim.DMBH, ls='', lw=0.75, color='forestgreen', marker='.')
# plt.scatter(maser_upplim.SIG, maser_upplim.MBH, color='forestgreen', marker='v', s=50)
# plt.errorbar(maser_true.SIG, maser_true.MBH, maser_true.DMBH, ls='', lw=0.75, color='forestgreen', marker='.', label='maser')
# plt.scatter(maser_true.SIG, maser_true.MBH, color='forestgreen', s=36)


# plt.scatter(omitted_upplim.SIG, omitted_upplim.MBH, color='grey', marker='v', s=50)
# plt.errorbar(omitted_true.SIG, omitted_true.MBH, ls='', lw=0.75, color='grey', marker='.', label='omitted')
# plt.scatter(omitted_true.SIG, omitted_true.MBH, color='grey', s=36)

# # Customizing legend to add upper limit marker and remove error bars
# ax = plt.gca()

# ax.text(1.66, 1.70, '$\\log(M_{BH}) = p(%.2f^{\\pm%.1f}+%.2f^{\\pm%.1f}\\log(\\sigma))$' % (gamma0, gamma0_err, gamma1, gamma1_err), fontsize=21)
# ax.text(1.87, 0.70, '$p = \\frac{1}{1+e^{-(%.2f^{\\pm%.1f}+%.2f^{\\pm%.2f}\\log(\\sigma))}}$' % (beta0, beta0_err, beta1, beta1_err), fontsize=22)

# # Customizing font size for axis labels
# ax.set_xlabel(r'velocity dispersion $\sigma$ (log km/s)', fontsize=25)
# ax.set_ylabel(r'BH mass $M_\bullet$ (log $M_\odot$)', fontsize=25)

# handles, labels = ax.get_legend_handles_labels()

# new_handles = []

# for h in handles:
#     # only needed to edit the errorbar legend entries
#     if isinstance(h, container.ErrorbarContainer):
#         new_handles.append(h[0])
#     else:
#         new_handles.append(h)
        
# # Customizing tick marks
# ax.tick_params(reset=True)
# ax.tick_params(which='major', direction='in', length=7, top=True, right=True, labelsize=20)
# ax.tick_params(which='minor', direction='in', length=2.5, top=True, right=True)
# ax.xaxis.set_major_locator(MultipleLocator(0.5))
# ax.xaxis.set_major_formatter('{x:.1f}')
# ax.xaxis.set_minor_locator(MultipleLocator(0.1))

# ax.yaxis.set_major_locator(MultipleLocator(2))
# ax.yaxis.set_major_formatter('{x:.0f}')
# ax.yaxis.set_minor_locator(MultipleLocator(0.5))

# # Remove tick label at beginning of x axis
# xticks = ax.xaxis.get_major_ticks()
# xticks[1].set_visible(False)

# # 95% credible interval
# labels.insert(2, 'Density') # modify the 2 depending on how many plots
# new_handles.insert(2, PolyCollection(verts=np.zeros((1,2,2)), color='lightgrey'))

# # upper limit marker
# labels.insert(3, 'upper limit') # modify the 3 depending on how many plots
# new_handles.insert(3, Line2D([0], [0], color='black', marker='v', ls='', markersize=4))


# ax.legend(new_handles, labels, markerscale=2.5, loc='upper left', fontsize='large', frameon=False)


# # Plotting and saving
# plt.savefig('../Figures/test.png')
# plt.show()

In [11]:
# # Velocity dispersion (sigma) values for plotting
# sigma_plot_values = np.linspace(0, 5, num=100)

# # Calculating fit with confidence interval
# fitted_line = hurdle(sigma_plot_values, beta0, beta1, gamma0, gamma1)
# y_2016 = -4 + 5.35*sigma_plot_values # coefficients from Remco Van Den Bosch 2016
# linear_portion = linear(sigma_plot_values, gamma0, gamma1)

# # Plot labels
# fig = plt.figure(figsize=(10,8), layout='constrained')
# plt.xlim(0, 6)
# # plt.ylim(-0.5, 11)

# # Plotting average fit with previous ones
# plt.plot(sigma_plot_values, fitted_line, color='black', label='Hurdle Fit')
# plt.plot(sigma_plot_values, linear_portion, color='red', linewidth=1.4, linestyle='dotted')
# #plt.plot(sigma_plot_values, hurdle(sigma_plot_values, -17.43, 9, 0.49, 0.7), color='black', linestyle='dashed', label='Without uncertainties') # coefficients from previous fit (without uncertainties)
# # plt.plot(sigma_plot_values, y_2016, color='black', linestyle='dotted', label='van den Bosch 2016')
# # plt.plot(sigma_plot_values, y_2016+0.49, color='dimgray', linestyle='dashed', alpha=0.7)
# # plt.plot(sigma_plot_values, y_2016-0.49, color='dimgray', linestyle='dashed', alpha=0.7)
# # plt.plot(sigma_plot_values, y_2016+3*0.49, color='dimgray', linestyle='dotted', alpha=0.7)
# # plt.plot(sigma_plot_values, y_2016-3*0.49, color='dimgray', linestyle='dotted', alpha=0.7)

# # Plotting data
# plt.errorbar(stars_upplim.SIG, stars_upplim.MBH, stars_upplim.DMBH, ls='', lw=0.75, color='darkred', marker='.')
# plt.scatter(stars_upplim.SIG, stars_upplim.MBH, color='darkred', marker='v', s=50)
# plt.errorbar(stars_true.SIG, stars_true.MBH, stars_true.DMBH, ls='', lw=0.75, color='darkred', marker='.', label='stars')
# plt.scatter(stars_true.SIG, stars_true.MBH, color='darkred', s=36)

# plt.scatter(stars_upplim_zero.SIG, stars_upplim_zero.MBH, color='darkred', marker='v', s=50)

# plt.errorbar(gas_upplim.SIG, gas_upplim.MBH, gas_upplim.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.')
# plt.scatter(gas_upplim.SIG, gas_upplim.MBH, color='dodgerblue', marker='v', s=50)
# plt.errorbar(gas_true.SIG, gas_true.MBH, gas_true.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.', label='gas')
# plt.scatter(gas_true.SIG, gas_true.MBH, color='dodgerblue', s=36)

# plt.errorbar(reverb_upplim.SIG, reverb_upplim.MBH, reverb_upplim.DMBH, ls='', lw=0.75, color='black', marker='.')
# plt.scatter(reverb_upplim.SIG, reverb_upplim.MBH, color='black', marker='v', s=50)
# plt.errorbar(reverb_true.SIG, reverb_true.MBH, reverb_true.DMBH, ls='', lw=0.75, color='black', marker='.', label='reverberation')
# plt.scatter(reverb_true.SIG, reverb_true.MBH, color='black', s=36)

# plt.errorbar(maser_upplim.SIG, maser_upplim.MBH, maser_upplim.DMBH, ls='', lw=0.75, color='forestgreen', marker='.')
# plt.scatter(maser_upplim.SIG, maser_upplim.MBH, color='forestgreen', marker='v', s=50)
# plt.errorbar(maser_true.SIG, maser_true.MBH, maser_true.DMBH, ls='', lw=0.75, color='forestgreen', marker='.', label='maser')
# plt.scatter(maser_true.SIG, maser_true.MBH, color='forestgreen', s=36)


# plt.scatter(omitted_upplim.SIG, omitted_upplim.MBH, color='grey', marker='v', s=50)
# plt.errorbar(omitted_true.SIG, omitted_true.MBH, ls='', lw=0.75, color='grey', marker='.', label='omitted')
# plt.scatter(omitted_true.SIG, omitted_true.MBH, color='grey', s=36)

# plt.savefig('../Figures/test.png')
# plt.show()

In [12]:
# brmsdf = np.loadtxt('./brms_conditional_effects.txt', delimiter=" ", dtype=float)
# cond_x = brmsdf[:, 0]
# cond_y = brmsdf[:, 1]
# cond_lower = brmsdf[:, 2]
# cond_upper = brmsdf[:, 3]

In [13]:
# pydf = np.loadtxt('./stan_samples_full_linear.txt', delimiter=" ", dtype=float)
# n_samples = pydf.shape[0]
# n_chains = 4
# steps = np.arange(n_samples//n_chains)

# sigma_plot_values = np.linspace(1, 2.8, num=100)
# y = np.empty((len(pydf), 100))

# for i in range(len(pydf)):
#     y[i] = hurdle(sigma_plot_values, pydf[i][0], pydf[i][1], pydf[i][2], pydf[i][3])

# stan_mean_full = np.median(y, axis=0)
# stan_lower_full = np.quantile(y, 0.975, axis=0)
# stan_upper_full = np.quantile(y, 0.025, axis=0)

# # Split chains
# chain1 = pydf[n_samples//n_chains * 0:n_samples//n_chains * 1, :]
# chain2 = pydf[n_samples//n_chains * 1:n_samples//n_chains * 2, :]
# chain3 = pydf[n_samples//n_chains * 2:n_samples//n_chains * 3, :]
# chain4 = pydf[n_samples//n_chains * 3:n_samples//n_chains * 4, :]

# #################################
# # Create subplots for first param
# fig, axs = plt.subplots(4, 1, sharex=True, figsize=(8, 10))

# # Plot data on each subplot
# axs[0].plot(steps, chain1[:, 0], label='Chain 1')
# axs[1].plot(steps, chain2[:, 0], label='Chain 2')
# axs[2].plot(steps, chain3[:, 0], label='Chain 3')
# axs[3].plot(steps, chain4[:, 0], label='Chain 4')

# axs[0].hlines(np.mean(chain1[:, 0]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain1[:, 0]):.2f}', linestyle='dashed')
# axs[1].hlines(np.mean(chain2[:, 0]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain2[:, 0]):.2f}', linestyle='dashed')
# axs[2].hlines(np.mean(chain3[:, 0]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain3[:, 0]):.2f}', linestyle='dashed')
# axs[3].hlines(np.mean(chain4[:, 0]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain4[:, 0]):.2f}', linestyle='dashed')

# # Add labels and legends
# axs[0].set_ylabel(r'$\beta_0$')
# axs[1].set_ylabel(r'$\beta_0$')
# axs[2].set_ylabel(r'$\beta_0$')
# axs[3].set_ylabel(r'$\beta_0$')

# axs[3].set_xlabel('Step')

# # Add legend to the last subplot
# axs[0].legend(loc='upper right')
# axs[1].legend(loc='upper right')
# axs[2].legend(loc='upper right')
# axs[3].legend(loc='upper right')
# axs[0].set_title(r'Chain convergence for $\beta_0$')
# plt.show()

# ##################################
# # Create subplots for second param
# fig, axs = plt.subplots(4, 1, sharex=True, figsize=(8, 10))

# # Plot data on each subplot
# axs[0].plot(steps, chain1[:, 1], label='Chain 1')
# axs[1].plot(steps, chain2[:, 1], label='Chain 2')
# axs[2].plot(steps, chain3[:, 1], label='Chain 3')
# axs[3].plot(steps, chain4[:, 1], label='Chain 4')

# axs[0].hlines(np.mean(chain1[:, 1]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain1[:, 1]):.2f}', linestyle='dashed')
# axs[1].hlines(np.mean(chain2[:, 1]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain2[:, 1]):.2f}', linestyle='dashed')
# axs[2].hlines(np.mean(chain3[:, 1]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain3[:, 1]):.2f}', linestyle='dashed')
# axs[3].hlines(np.mean(chain4[:, 1]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain4[:, 1]):.2f}', linestyle='dashed')

# # Add labels and legends
# axs[0].set_ylabel(r'$\beta_1$')
# axs[1].set_ylabel(r'$\beta_1$')
# axs[2].set_ylabel(r'$\beta_1$')
# axs[3].set_ylabel(r'$\beta_1$')

# axs[3].set_xlabel('Step')

# # Add legend to the last subplot
# axs[0].legend(loc='upper right')
# axs[1].legend(loc='upper right')
# axs[2].legend(loc='upper right')
# axs[3].legend(loc='upper right')
# axs[0].set_title(r'Chain convergence for $\beta_1$')
# plt.show()

# #################################
# # Create subplots for third param
# fig, axs = plt.subplots(4, 1, sharex=True, figsize=(8, 10))

# # Plot data on each subplot
# axs[0].plot(steps, chain1[:, 2], label='Chain 1')
# axs[1].plot(steps, chain2[:, 2], label='Chain 2')
# axs[2].plot(steps, chain3[:, 2], label='Chain 3')
# axs[3].plot(steps, chain4[:, 2], label='Chain 4')

# axs[0].hlines(np.mean(chain1[:, 2]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain1[:, 2]):.2f}', linestyle='dashed')
# axs[1].hlines(np.mean(chain2[:, 2]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain2[:, 2]):.2f}', linestyle='dashed')
# axs[2].hlines(np.mean(chain3[:, 2]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain3[:, 2]):.2f}', linestyle='dashed')
# axs[3].hlines(np.mean(chain4[:, 2]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain4[:, 2]):.2f}', linestyle='dashed')

# # Add labels and legends
# axs[0].set_ylabel(r'$\gamma_0$')
# axs[1].set_ylabel(r'$\gamma_0$')
# axs[2].set_ylabel(r'$\gamma_0$')
# axs[3].set_ylabel(r'$\gamma_0$')

# axs[3].set_xlabel('Step')

# # Add legend to the last subplot
# axs[0].legend(loc='upper right')
# axs[1].legend(loc='upper right')
# axs[2].legend(loc='upper right')
# axs[3].legend(loc='upper right')
# axs[0].set_title(r'Chain convergence for $\gamma_0$')
# plt.show()

# ##################################
# # Create subplots for fourth param
# fig, axs = plt.subplots(4, 1, sharex=True, figsize=(8, 10))

# # Plot data on each subplot
# axs[0].plot(steps, chain1[:, 3], label='Chain 1')
# axs[1].plot(steps, chain2[:, 3], label='Chain 2')
# axs[2].plot(steps, chain3[:, 3], label='Chain 3')
# axs[3].plot(steps, chain4[:, 3], label='Chain 4')

# axs[0].hlines(np.mean(chain1[:, 3]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain1[:, 3]):.2f}', linestyle='dashed')
# axs[1].hlines(np.mean(chain2[:, 3]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain2[:, 3]):.2f}', linestyle='dashed')
# axs[2].hlines(np.mean(chain3[:, 3]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain3[:, 3]):.2f}', linestyle='dashed')
# axs[3].hlines(np.mean(chain4[:, 3]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain4[:, 3]):.2f}', linestyle='dashed')

# # Add labels and legends
# axs[0].set_ylabel(r'$\gamma_1$')
# axs[1].set_ylabel(r'$\gamma_1$')
# axs[2].set_ylabel(r'$\gamma_1$')
# axs[3].set_ylabel(r'$\gamma_1$')

# axs[3].set_xlabel('Step')

# # Add legend to the last subplot
# axs[0].legend(loc='upper right')
# axs[1].legend(loc='upper right')
# axs[2].legend(loc='upper right')
# axs[3].legend(loc='upper right')
# axs[0].set_title(r'Chain convergence for $\gamma_1$')
# plt.show()

In [14]:
pydf = np.loadtxt('./final_model.txt', delimiter=" ", dtype=float)

sigma_plot_values = np.linspace(1, 2.8, num=100)
y = np.empty((len(pydf), 100))
lin = np.empty((len(pydf), 100))

for i in range(len(pydf)):
    y[i] = hurdle(sigma_plot_values, pydf[i][0], pydf[i][1], pydf[i][2], pydf[i][3])
    lin[i] = linear(sigma_plot_values, pydf[i][2], pydf[i][3])

stan_mean = np.median(y, axis=0)
stan_lower = np.quantile(y, 0.975, axis=0)
stan_upper = np.quantile(y, 0.025, axis=0)

lin_mean = np.median(lin, axis=0)

np.mean(pydf, axis=0)

array([-4.42003547,  4.18940012, -2.88235774,  4.90000327])

In [15]:
# pydf = np.loadtxt('./stan_samples.txt', delimiter=" ", dtype=float)

# sigma_plot_values = np.linspace(1, 2.8, num=100)
# y = np.empty((len(pydf), 100))

# for i in range(len(pydf)):
#     y[i] = hurdle_brms(sigma_plot_values, pydf[i][0], pydf[i][1], pydf[i][2], pydf[i][3])

# stan_exp_mean = np.median(y, axis=0)
# stan_exp_lower = np.quantile(y, 0.975, axis=0)
# stan_exp_upper = np.quantile(y, 0.025, axis=0)

In [16]:
# pydf = np.loadtxt('./brms_samples.txt', delimiter=" ", dtype=float)

# sigma_plot_values = np.linspace(1, 2.8, num=100)
# y = np.empty((len(pydf), 100))

# for i in range(len(pydf)):
#     y[i] = hurdle_brms(sigma_plot_values, pydf[i][0], pydf[i][1], pydf[i][2], pydf[i][3])

# brms_mean = np.median(y, axis=0)
# brms_lower = np.quantile(y, 0.975, axis=0)
# brms_upper = np.quantile(y, 0.025, axis=0)

In [17]:
# Velocity dispersion (sigma) values for plotting
sigma_plot_values = np.linspace(1, 2.8, num=100)

# Calculating fit with confidence interval
# fitted_line = hurdle(sigma_plot_values, beta0, beta1, gamma0, gamma1)
# y_2016 = -4 + 5.35*sigma_plot_values # coefficients from Remco Van Den Bosch 2016
# linear_portion = linear(sigma_plot_values, gamma0, gamma1)

# Plot labels
fig = plt.figure(figsize=(10,8), layout='constrained')
plt.xlim(1.0, 2.8)
plt.ylim(-0.5, 11)

# Plotting average fit with previous ones
# plt.plot(sigma_plot_values, stan_mean, color='black', label='Normal Likelihood')
# plt.fill_between(sigma_plot_values, stan_lower, stan_upper, alpha=0.3, color='black')

plt.plot(sigma_plot_values, stan_mean, color='black', label='Log-Normal Likelihood')
plt.fill_between(sigma_plot_values, stan_lower, stan_upper, alpha=0.15, color='black')

# plt.plot(sigma_plot_values, brms_mean, color='red', label='BRMS Posterior Samples', linestyle='dotted')
# plt.fill_between(sigma_plot_values, brms_lower, brms_upper, alpha=0.2, color='salmon')

# plt.plot(cond_x, cond_y, color='black', label='BRMS Conditional Effects')
# plt.fill_between(cond_x, cond_lower, cond_upper, alpha=0.7, color='lightgrey')

plt.plot(sigma_plot_values, lin_mean, color='black', linestyle='dashed', label='Linear Portion')
# plt.plot(sigma_plot_values, fitted_line, color='black', label='Hurdle Fit')
# plt.plot(sigma_plot_values, linear_portion, color='red', linewidth=1.4, linestyle='dotted')
#plt.plot(sigma_plot_values, hurdle(sigma_plot_values, -17.43, 9, 0.49, 0.7), color='black', linestyle='dashed', label='Without uncertainties') # coefficients from previous fit (without uncertainties)
# plt.plot(sigma_plot_values, y_2016, color='black', linestyle='dotted', label='van den Bosch 2016')
# plt.plot(sigma_plot_values, y_2016+0.49, color='dimgray', linestyle='dashed', alpha=0.7)
# plt.plot(sigma_plot_values, y_2016-0.49, color='dimgray', linestyle='dashed', alpha=0.7)
# plt.plot(sigma_plot_values, y_2016+3*0.49, color='dimgray', linestyle='dotted', alpha=0.7)
# plt.plot(sigma_plot_values, y_2016-3*0.49, color='dimgray', linestyle='dotted', alpha=0.7)

# Plotting all fits for confidence interval
# for i in range(len(fitted_params)):
#   plt.plot(sigma_plot_values, hurdle(sigma_plot_values, fitted_params[i][0], fitted_params[i][1], fitted_params[i][2], fitted_params[i][3]), alpha=0.003, color='lightgrey')

# for i in range(len(y)):
#     plt.plot(sigma_plot_values, y[i], alpha=0.003, color='lightgrey')

# Plotting data
plt.errorbar(stars_upplim.SIG, stars_upplim.MBH, stars_upplim.DMBH, ls='', lw=0.75, color='darkred', marker='.')
plt.scatter(stars_upplim.SIG, stars_upplim.MBH, color='darkred', marker='v', s=50)
plt.errorbar(stars_true.SIG, stars_true.MBH, stars_true.DMBH, ls='', lw=0.75, color='darkred', marker='.', label='stars')
plt.scatter(stars_true.SIG, stars_true.MBH, color='darkred', s=36)

plt.scatter(stars_upplim_zero.SIG, stars_upplim_zero.MBH, color='darkred', marker='v', s=50)

plt.errorbar(gas_upplim.SIG, gas_upplim.MBH, gas_upplim.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.')
plt.scatter(gas_upplim.SIG, gas_upplim.MBH, color='dodgerblue', marker='v', s=50)
plt.errorbar(gas_true.SIG, gas_true.MBH, gas_true.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.', label='gas')
plt.scatter(gas_true.SIG, gas_true.MBH, color='dodgerblue', s=36)

plt.errorbar(reverb_upplim.SIG, reverb_upplim.MBH, reverb_upplim.DMBH, ls='', lw=0.75, color='black', marker='.')
plt.scatter(reverb_upplim.SIG, reverb_upplim.MBH, color='black', marker='v', s=50)
plt.errorbar(reverb_true.SIG, reverb_true.MBH, reverb_true.DMBH, ls='', lw=0.75, color='black', marker='.', label='reverberation')
plt.scatter(reverb_true.SIG, reverb_true.MBH, color='black', s=36)

plt.errorbar(maser_upplim.SIG, maser_upplim.MBH, maser_upplim.DMBH, ls='', lw=0.75, color='forestgreen', marker='.')
plt.scatter(maser_upplim.SIG, maser_upplim.MBH, color='forestgreen', marker='v', s=50)
plt.errorbar(maser_true.SIG, maser_true.MBH, maser_true.DMBH, ls='', lw=0.75, color='forestgreen', marker='.', label='maser')
plt.scatter(maser_true.SIG, maser_true.MBH, color='forestgreen', s=36)


plt.scatter(omitted_upplim.SIG, omitted_upplim.MBH, color='grey', marker='v', s=50)
plt.errorbar(omitted_true.SIG, omitted_true.MBH, ls='', lw=0.75, color='grey', marker='.', label='omitted')
plt.scatter(omitted_true.SIG, omitted_true.MBH, color='grey', s=36)

# Customizing legend to add upper limit marker and remove error bars
ax = plt.gca()

# ax.text(1.66, 1.70, '$\\log(M_{BH}) = p(%.2f^{\\pm%.1f}+%.2f^{\\pm%.1f}\\log(\\sigma))$' % (gamma0, gamma0_err, gamma1, gamma1_err), fontsize=21)
# ax.text(1.87, 0.70, '$p = \\frac{1}{1+e^{-(%.2f^{\\pm%.1f}+%.2f^{\\pm%.2f}\\log(\\sigma))}}$' % (beta0, beta0_err, beta1, beta1_err), fontsize=22)

# Customizing font size for axis labels
ax.set_xlabel(r'velocity dispersion $\sigma$ (log km/s)', fontsize=25)
ax.set_ylabel(r'BH mass $M_\bullet$ (log $M_\odot$)', fontsize=25)

handles, labels = ax.get_legend_handles_labels()

new_handles = []

for h in handles:
    # only needed to edit the errorbar legend entries
    if isinstance(h, container.ErrorbarContainer):
        new_handles.append(h[0])
    else:
        new_handles.append(h)
        
# Customizing tick marks
ax.tick_params(reset=True)
ax.tick_params(which='major', direction='in', length=7, top=True, right=True, labelsize=20)
ax.tick_params(which='minor', direction='in', length=2.5, top=True, right=True)
ax.xaxis.set_major_locator(MultipleLocator(0.5))
ax.xaxis.set_major_formatter('{x:.1f}')
ax.xaxis.set_minor_locator(MultipleLocator(0.1))

ax.yaxis.set_major_locator(MultipleLocator(2))
ax.yaxis.set_major_formatter('{x:.0f}')
ax.yaxis.set_minor_locator(MultipleLocator(0.5))

# Remove tick label at beginning of x axis
xticks = ax.xaxis.get_major_ticks()
xticks[1].set_visible(False)

# 95% credible interval
# labels.insert(3, 'Density') # modify the 2 depending on how many plots
# new_handles.insert(3, PolyCollection(verts=np.zeros((1,2,2)), color='lightgrey'))

# upper limit marker
labels.insert(4, 'upper limit') # modify the 3 depending on how many plots
new_handles.insert(4, Line2D([0], [0], color='black', marker='v', ls='', markersize=4))


ax.legend(new_handles, labels, markerscale=2.5, loc='upper left', fontsize='large', frameon=False)


# Plotting and saving
plt.savefig('../../Figures/final_model.png')
plt.show()

libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast
